# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft


import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
import inspect
import IPython.display as dp
dp.Code(inspect.getsource(pbs_profile))

In [ ]:
hpc05.create_pbs_profile()

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
# hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(300, folder='two_dim_majoranas')

# Define and plot system

In [ ]:
syst_pars = {'L_down' :  200,
               'L_up' : 200,
               'L_m' : 200,
               'L_x' : 6.25,
               'a' :  6.25,
            'mu_from_bottom_of_spin_orbit_bands': True,
            'k_x_in_sc' : True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10.0,
                 alpha_middle = 19.05,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 0.0,
                 phase = np.pi/2,
                 T = 0.025,
                 V=0)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,4]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = { "energy_gap":{"tol":1e-4}
                     }

### Define data folder

In [ ]:
data_folder = 'what_is_infinite'

# Make ASS

In [ ]:
ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

### Add additional dimensions

### Chemical potential

In [ ]:
mus = [10]

def _params_mu(syst_pars, params, mu):
    for key in ['mu']:
        if key in params:
            params[key] = mu
            if key is 'V':
                params[key] = {70:35, 50:32, 10:14}[mu]
        if key in syst_pars:
            syst_pars[key] = mu
    return mu

params_mu = [ft.partial(_params_mu,
                           mu=_mu
                           ) for _mu in mus]

ass.add_dimension("mu", params_mu)

### Make learner

In [ ]:
ass.make_balancing_learner(1000)
ass.load(data_folder, 1000)


### Make runner with saver


In [ ]:
runner = adaptive.Runner(ass.get_balancing_learner())#, executor=client)
ass.start_periodic_saver(runner, data_folder, interval=180)
runner.live_info(update_interval=1)

In [ ]:
runner.task.result()

In [ ]:
sum([l.npoints for l in ass.learners])

In [ ]:
hv.notebook_extension('matplotlib')

In [ ]:
kdims, pdict = ass.get_plot_dict(800, tables=True)

In [ ]:
%%opts Image [colorbar=True logz=True fig_size=200] {+framewise}
%%output filename=f'./{data_folder}/gap_150' fig='png'
hv.HoloMap(pdict, kdims)

## import formulas as f
vf = f.fermi_velocity(10, 0.02)
kf = f.fermi_wavenumber(10, 0.02)
eth = f.thouless_energy(vf, 100)/(params['g_factor_middle'] * params['mu_B']) 
#- params['m_eff']*params['alpha_middle']**2/(2*params['hbar']**2)

In [ ]:
supercurrent.fermi_dirac(1e-2, dict(T=40e-3, k=params['k']))

# Plot

In [ ]:
70 35
50 32
10 14

In [ ]:
params['mu'] = 10
params['V'] = 14
syst = sns_system.make_ns_junction(**syst_pars)

In [ ]:
import scattering
scattering.transparency(syst, params)

In [ ]:
kdims, pdict = ass.get_plot_dict(200, False, False)

In [ ]:
hv.extension('bokeh')


In [ ]:
pdict = {(electron_density[idx]*1e3, k[1]): v * hv.Path((list(keys_with_bounds['phase']), (eth[idx], eth[idx]) )).options(color='red', line_width=4) for (idx, (k,v)) in enumerate(pdict.items())}

In [ ]:
%%opts Image [colorbar=True width=400 height=300 aspect=1] {+framewise}
# %%output filename=f'./{data_folder}/transmission' fig='png'
hv.HoloMap(pdict, kdims=['density', kdims[1]])

In [ ]:
kdims, pdict = ass.get_plot_dict(200, contour_pfaffian=True, tables=True)
pdict = {k: d.redim.range(z=(5e-3, 2.5e-2)) if k[2]=="energy_gap" else d for k,d in pdict.items()}

In [ ]:
constants['k']*0.05

In [ ]:
ass.learners[0].plot(tri_alpha=1)

In [ ]:
hv.extension('matplotlib')
# %%opts Image [colorbar=True]
pdict[(10, 250, 'energy_gap')].opts(style=dict(cmap='Viridis'))

In [ ]:
_=pdict[(10, 250, 'energy_gap')]

In [ ]:
_.opts(style=dict(cmap='Viridis'), plot=dict(colorbar=True, aspect=1))